In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm

In [26]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# 데이터 전처리 (CIFAR10은 3채널 이미지)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# CIFAR10 학습 및 테스트 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# CIFAR10의 targets는 리스트로 되어 있으므로 tensor로 변환
targets = torch.tensor(train_dataset.targets)

num_samples_per_class = 500
selected_indices = []

# 0부터 9까지 각 클래스별로 인덱스를 추출하고, 랜덤하게 num_samples_per_class개 선택
for class_label in range(10):
    # 해당 클래스의 인덱스 추출
    indices = (targets == class_label).nonzero(as_tuple=True)[0]
    # 인덱스를 랜덤하게 섞은 후 원하는 개수만 선택
    selected = indices[torch.randperm(len(indices))[:num_samples_per_class]]
    selected_indices.extend(selected.tolist())

# 선택한 인덱스만 남긴 서브셋 생성
train_dataset = Subset(train_dataset, selected_indices)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [27]:
# DATASET : MNIST -> input_size = (24, 24, 1)
# K = 2P + 1 to make the output size same as the input size
class SigmoidCNN(nn.Module):
    def __init__(self, num_blocks):
        super(SigmoidCNN, self).__init__()
        def build_block():
            return nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(64),
                nn.Sigmoid()
            )

        self.num_blocks = num_blocks
        self.base = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.Sigmoid())
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.blocks = nn.ModuleList([build_block() for _ in range(self.num_blocks)])
        self.classifier = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = self.base(x)
        for block in self.blocks:
            x = block(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class ReLUCNN(nn.Module):
    def __init__(self, num_blocks):
        super(ReLUCNN, self).__init__()
        def build_block():
            return nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU()
            )
        self.num_blocks = num_blocks
        self.base = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.blocks = nn.ModuleList([build_block() for _ in range(self.num_blocks)])
        self.classifier = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = self.base(x)
        for block in self.blocks:
            x = block(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(8):
    models = (model1:=SigmoidCNN(i), model2:=ReLUCNN(i))
    print(f"ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: {i+1}ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
    for model in models:
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        total_loss = 0.0
        #Train Code
        for epoch in range(5):
            model.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

    for model in models:
        #Test Code
        model.to(device)
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = data.to(device), target.to(device)
                output = model(data) # size = (64, 10)
                correct += (output.argmax(dim=1) == target).sum().item()
        top1_acc = 100 * correct / len(test_loader.dataset)
        print(f"Model : {model.__class__.__name__}Block Number: {i+1}, Top1 Accuracy: {top1_acc:.2f}%")
    print(f"ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: {i+1}ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")


ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 1ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
Model : SigmoidCNNBlock Number: 1, Top1 Accuracy: 21.15%
Model : ReLUCNNBlock Number: 1, Top1 Accuracy: 25.71%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 1ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 2ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
Model : SigmoidCNNBlock Number: 2, Top1 Accuracy: 22.95%
Model : ReLUCNNBlock Number: 2, Top1 Accuracy: 33.29%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 2ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 3ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
Model : SigmoidCNNBlock Number: 3, Top1 Accuracy: 21.49%
Model : ReLUCNNBlock Number: 3, Top1 Accuracy: 37.71%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 3ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 4ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
Model : SigmoidCNNBlock Number: 4, Top1 Accuracy: 23.55%
Model : ReLUCNNBlock Number: 4, Top1 Accuracy: 34.35%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 4ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 5ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
Model : SigmoidCNNBlock Number: 5, Top1 Accuracy: 23.38%
Model : ReLUCNNBlock Number: 5, Top1 Accuracy: 35.40%
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 5ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡBlock Number: 6ㅡㅡㅡ